# Mapping Planter & Harvester GPS coordinates

It is commonplace for agricultural data to contain spatial information typically the longitude and latitude at which each data point was collected. However, one of the challenges this introduces is that different datasets often do not contain data that were collected at the same locations. A specific example involves planter and harvester data from precision farming equipment. As a planter moves through a field, it records thousands of measurements of its location along with various data about the planter operation (such as the variety of seed planted, the seeding rate, and the spacing between seeds). Similarly, as a harvester moves through a field, it records thousands of measurements of its location and the crop’s yield at that point. A question of agricultural interest is to estimate the relationship between yield and the variables measured during planter. However, the harvester data points do not necessarily exactly overlap the planter data points, so before any analysis can be conducted to probe these relationships, it’s necessary to determine which planter points are associated with which harvesterpoints.

Let's devise an algorithm to efficiently associate the planter data points with the harvester data points for one corn field (from the same year).

## Import packages

In [1]:
import pandas as pd
import numpy as np
import math
from haversine import haversine
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm import tqdm
pd.set_option('precision', 11)

## Import data

The files contain a row for each data point collected by the planter and harvester machines. The columns in each file are as follows:

**planter_data.csv**
1. **long**: longitude where data point was collected.
2. **lat**: latitude where data point was collected.
3. **variety**: the seed variety planted at that location.
4. **seeding_rate**: continuous variable specifying the number of seeds planted per acre (in thousands).
5. **seed_spacing**: continuous variable specifying the distance between seeds (inches).

**harvester_data.csv**
1. **long**: longitude where data point was collected
2. **lat**: latitude where data point was collected
3. **yield**: continuous variable specifying the yield of the crop (in bushels/acre).

In [2]:
planter = pd.read_csv('planter_data.csv')
harvester = pd.read_csv('harvester_data.csv')
planter = planter.sort_values(['long','lat']).reset_index(drop=True)
harvester = harvester.sort_values(['long','lat']).reset_index(drop=True)

In [3]:
planter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6314 entries, 0 to 6313
Data columns (total 6 columns):
long            6314 non-null float64
lat             6314 non-null float64
variety         6314 non-null object
seeding_rate    6314 non-null float64
seed_spacing    6314 non-null float64
speed           6314 non-null float64
dtypes: float64(5), object(1)
memory usage: 296.0+ KB


In [4]:
harvester.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16626 entries, 0 to 16625
Data columns (total 3 columns):
long     16626 non-null float64
lat      16626 non-null float64
yield    16626 non-null float64
dtypes: float64(3)
memory usage: 389.8 KB


In [5]:
planter.describe()

,long,lat,seeding_rate,seed_spacing,speed
count,6314.00000000000,6314.00000000000,6314.00000000000,6314.00000000000,6314.00000000000
mean,-91.84829859862,40.37765799823,36.69490574913,5.70821582198,4.57630503643
std,0.00136728733,0.00104623597,2.52255031654,1.02914093892,0.52876202165
min,-91.85061399968,40.37588045151,0.00000000000,0.00000000000,0.02000000000
25%,-91.84944822468,40.37674532651,35.84910000000,5.54350000000,4.50000000000
50%,-91.84828204968,40.37765820151,37.26410000000,5.61080000000,4.75000000000
75%,-91.84711422468,40.37857565151,37.70820000000,5.82617500000,4.90000000000
max,-91.84597999968,40.37939005151,48.81620000000,45.57280000000,5.19000000000


In [6]:
harvester.describe()

,long,lat,yield
count,16626.00000000000,16626.00000000000,16626.00000000000
mean,-91.84833560319,40.37767474760,231.15184726913
std,0.00134872697,0.00101866436,26.59249177464
min,-91.85065794214,40.37588862999,42.42315244164
25%,-91.84948667615,40.37680786733,219.77168948374
50%,-91.84834097814,40.37767991793,231.87076628901
75%,-91.84717053079,40.37855887688,244.17457385903
max,-91.84594757114,40.37942139599,365.70835825733


In [7]:
planter.head()

,long,lat,variety,seeding_rate,seed_spacing,speed
0,-91.85061399968,40.37588045151,DKC63-33RIB,0.0000,0.0000,0.02
1,-91.85061239968,40.37596835151,DKC63-33RIB,0.0000,0.0000,3.75
2,-91.85061169968,40.37600165151,DKC63-33RIB,0.0000,0.0000,3.76
3,-91.85061139968,40.37602595151,DKC63-33RIB,31.7371,6.5881,3.77
4,-91.85061069968,40.37605615151,DKC63-33RIB,32.8151,6.3717,3.76


In [8]:
harvester.head()

,long,lat,yield
0,-91.85065794214,40.37589336599,216.92110846492
1,-91.85065682914,40.37590353899,218.21178574100
2,-91.85065636914,40.37591472799,241.78874779943
3,-91.85065582614,40.37592657599,227.94857084442
4,-91.85065529614,40.37593849799,226.69384423697


### Bounding box & field dimensions

In [9]:
b_long = min(planter['long'].min(),harvester['long'].min()),max(planter['long'].max(),harvester['long'].max())
print (b_long)
b_lat = min(planter['lat'].min(),harvester['lat'].min()),max(planter['lat'].max(),harvester['lat'].max())
print (b_lat)

(-91.850657942140501, -91.8459475711404)
(40.375880451506298, 40.379421395990498)


In [10]:
length = round(haversine((b_lat[0], b_long[0]),(b_lat[1], b_long[0]))*1000,1)
print (length, ' m')
breadth = round(haversine((b_lat[0], b_long[0]),(b_lat[0], b_long[1]))*1000,1)
print (breadth, ' m')

393.7  m
399.0  m


In [11]:
lat_pm = (b_lat[1]-b_lat[0])/length
print (lat_pm)
long_pm = (b_long[1]-b_long[0])/breadth
print (long_pm)

8.9940169779e-06
1.1805441103e-05


### Plot

In [12]:
plt.rcParams['figure.figsize'] = (32.0, 32.0)
_= plt.scatter(planter['long'], planter['lat'], alpha=0.5)
_= plt.scatter(harvester['long'], harvester['lat'], alpha=0.5, c='r')
plt.savefig('images/planter-harvester-py.png')

![Planter-Harvester](images/planter-harvester-py.png)

### Create Gridlines

In [13]:
def myrange(start, step, count):
    return [start+i*step for i in range(count)]

In [14]:
# vertical gridlines
nx = math.ceil((breadth+(long_pm * 5))/10)
x = myrange(b_long[0]-(long_pm * 5), (long_pm * 10), count = nx+2)
print ("Vertical Gridlines: ", len(x))
# horizontal gridlines 
ny = math.ceil((breadth+(lat_pm * 5))/10)
y = myrange(b_lat[0]-(lat_pm * 5), (lat_pm * 10), count = ny+2)
print ("Horizontal Gridlines: ", len(y))

Vertical Gridlines:  42
Horizontal Gridlines:  42


### Create boundaries for Grids

In [15]:
# longitude range
dfx = pd.DataFrame(x)
dfx.columns = ['value']
dfx['value1'] = dfx.shift(-1)-1e-10
dfx['groupX'] = range(0,len(dfx['value']),1)
dfx = dfx.dropna()
dfx.info()

# latitude range
dfy = pd.DataFrame(y)
dfy.columns = ['value']
dfy['value1'] = dfy.shift(-1)-1e-10
dfy['groupY'] = range(0,len(dfy['value']),1)
dfy = dfy.dropna()
dfy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 0 to 40
Data columns (total 3 columns):
value     41 non-null float64
value1    41 non-null float64
groupX    41 non-null int64
dtypes: float64(2), int64(1)
memory usage: 1.3 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 0 to 40
Data columns (total 3 columns):
value     41 non-null float64
value1    41 non-null float64
groupY    41 non-null int64
dtypes: float64(2), int64(1)
memory usage: 1.3 KB


In [16]:
# Check lower boundary < upper boundary
print (sum(dfx['value']< dfx['value1']))
print (sum(dfy['value']< dfy['value1']))

41
41


### Assign Grid Membership

In [17]:
import itertools

def expand_grid(data_dict):
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

group = expand_grid({'groupX': range(0, dfx.count()[0], 1),
        'groupY': range(0, dfy.count()[0], 1)})

group['value'] = group['groupY'].map(str) + "-" + group['groupX'].map(str)
group['group'] = range(0, group.count()[0], 1)
group.drop(['groupX','groupY'],axis=1, inplace=True)
group.head()

,value,group
0,0-0,0
1,0-1,1
2,0-2,2
3,0-3,3
4,0-4,4


### Map planter data to grids

In [18]:
# vertical grids
planter_grid = planter.copy()
planter_grid['groupX'] = np.piecewise(np.zeros(planter_grid.count()[0]),
                                 [(planter_grid['long'].values >= start) & (planter_grid['long'].values <= end) for start, end in zip(dfx['value'].values, dfx['value1'].values)], dfx['groupX'].values).astype(int) 
# horizontal grids
planter_grid['groupY'] = np.piecewise(np.zeros(planter.count()[0]),
                                 [(planter_grid['lat'].values >= start) & (planter_grid['lat'].values <= end) for start, end in zip(dfy['value'].values, dfy['value1'].values)], dfy['groupY'].values).astype(int)

# grid membership
planter_grid['value'] = planter_grid['groupY'].map(str) + "-" + planter_grid['groupX'].map(str)
planter_grid.drop(['groupX','groupY'],axis=1, inplace=True)
planter_grid = pd.merge(planter_grid, group, on='value')
planter_grid = planter_grid.sort_values(['long','lat']).reset_index(drop=True)
planter_grid.head()

,long,lat,variety,seeding_rate,seed_spacing,speed,value,group
0,-91.85061399968,40.37588045151,DKC63-33RIB,0.0000,0.0000,0.02,0-0,0
1,-91.85061239968,40.37596835151,DKC63-33RIB,0.0000,0.0000,3.75,1-0,41
2,-91.85061169968,40.37600165151,DKC63-33RIB,0.0000,0.0000,3.76,1-0,41
3,-91.85061139968,40.37602595151,DKC63-33RIB,31.7371,6.5881,3.77,2-0,82
4,-91.85061069968,40.37605615151,DKC63-33RIB,32.8151,6.3717,3.76,2-0,82


In [19]:
# Check for NULL values
planter_grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6314 entries, 0 to 6313
Data columns (total 8 columns):
long            6314 non-null float64
lat             6314 non-null float64
variety         6314 non-null object
seeding_rate    6314 non-null float64
seed_spacing    6314 non-null float64
speed           6314 non-null float64
value           6314 non-null object
group           6314 non-null int64
dtypes: float64(5), int64(1), object(2)
memory usage: 394.7+ KB


In [20]:
# Check for data consistency
planter_grid.loc[:,['long','lat','variety','seeding_rate','seed_spacing','speed']].equals(planter)

True

### Map harvester data to grids

In [21]:
#vertical grids
harvester_grid = harvester.copy()
harvester_grid['groupX'] = np.piecewise(np.zeros(harvester_grid.count()[0]),
                                 [(harvester_grid['long'].values >= start) & (harvester_grid['long'].values <= end) for start, end in zip(dfx['value'].values, dfx['value1'].values)], dfx['groupX'].values).astype(int) 
#horizontal grids
harvester_grid['groupY'] = np.piecewise(np.zeros(harvester_grid.count()[0]),
                                 [(harvester_grid['lat'].values >= start) & (harvester_grid['lat'].values <= end) for start, end in zip(dfy['value'].values, dfy['value1'].values)], dfy['groupY'].values).astype(int)

# grid membership
harvester_grid['value'] = harvester_grid['groupY'].map(str) + "-" + harvester_grid['groupX'].map(str)
harvester_grid.drop(['groupX','groupY'],axis=1, inplace=True)
harvester_grid = pd.merge(harvester_grid, group, on='value')
harvester_grid = harvester_grid.sort_values(['long','lat']).reset_index(drop=True)
harvester_grid.head()

,long,lat,yield,value,group
0,-91.85065794214,40.37589336599,216.92110846492,0-0,0
1,-91.85065682914,40.37590353899,218.21178574100,0-0,0
2,-91.85065636914,40.37591472799,241.78874779943,0-0,0
3,-91.85065582614,40.37592657599,227.94857084442,1-0,41
4,-91.85065529614,40.37593849799,226.69384423697,1-0,41


In [22]:
# Check for NULL values
harvester_grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16626 entries, 0 to 16625
Data columns (total 5 columns):
long     16626 non-null float64
lat      16626 non-null float64
yield    16626 non-null float64
value    16626 non-null object
group    16626 non-null int64
dtypes: float64(3), int64(1), object(1)
memory usage: 649.5+ KB


In [23]:
# Check for data consistency
harvester_grid.loc[:,['long','lat','yield']].equals(harvester)

True

### Map planter - harvester data

Algorithm: Nearest neigbour search

In [24]:
def haversine_vect(x):
    return round(haversine((x['lat'], x['long']),(x['lat1'], x['long1']))*1000,1)

In [25]:
harvester_grid = pd.concat([harvester_grid,
                            pd.DataFrame(columns=['variety','seeding_rate','seed_spacing','speed','long1','lat1','dist'])], axis=1)

In [26]:
for i in tqdm(range(harvester_grid.count()[0])):
    harvester_gps = harvester_grid.loc[i,].copy()
    n = harvester_gps['value'].split('-')
    row = int(n[0])
    col = int(n[1])
              
    # filter the rows from planting - nearest neighbour search      
    neighbour = ["%s-%s" %(row, col-1),
                  "%s-%s" %(row-1, col),
                  "%s-%s" %(row, col),
                  "%s-%s" %(row+1, col),
                  "%s-%s" %(row, col+1)]
    
    planter_gps = planter_grid[planter_grid['value'].isin(neighbour)].copy().reset_index(drop=True)
    
    if(planter_gps.count()[0] >0):
        planter_gps['long1'] = harvester_gps['long']
        planter_gps['lat1'] = harvester_gps['lat']
        # find the distance between geo-coordinates
        planter_gps['dist'] = planter_gps.apply(lambda row: haversine_vect(row), axis=1)
        planter_gps = planter_gps[planter_gps['dist'] == planter_gps['dist'].min()].reset_index(drop=True)
        
        # map the data back to harvester_grid
        harvester_grid.loc[i,'variety'] = planter_gps.loc[0,'variety']
        harvester_grid.loc[i,'seeding_rate'] = planter_gps.loc[0,'seeding_rate']
        harvester_grid.loc[i,'seed_spacing'] = planter_gps.loc[0,'seed_spacing']
        harvester_grid.loc[i,'speed'] = planter_gps.loc[0,'speed']
        harvester_grid.loc[i,'long1'] = planter_gps.loc[0,'long1']
        harvester_grid.loc[i,'lat1'] = planter_gps.loc[0,'lat1']
        harvester_grid.loc[i,'dist'] = planter_gps.loc[0,'dist']

In [27]:
harvester_grid.head()

,long,lat,yield,value,group,variety,seeding_rate,seed_spacing,speed,long1,lat1,dist
0,-91.85065794214,40.37589336599,216.92110846492,0-0,0,DKC63-33RIB,0,0,0.02,-91.850657942,40.375893366,4
1,-91.85065682914,40.37590353899,218.21178574100,0-0,0,DKC63-33RIB,0,0,0.02,-91.850656829,40.375903539,4.4
2,-91.85065636914,40.37591472799,241.78874779943,0-0,0,DKC63-33RIB,0,0,0.02,-91.850656369,40.375914728,5.2
3,-91.85065582614,40.37592657599,227.94857084442,1-0,41,DKC63-33RIB,0,0,3.75,-91.850655826,40.375926576,5.9
4,-91.85065529614,40.37593849799,226.69384423697,1-0,41,DKC63-33RIB,0,0,3.75,-91.850655296,40.375938498,4.9


### Examine the distribution of distance between planter and harvester gps coordinates

In [ ]:
plt.rcParams['figure.figsize'] = (8.0, 4.0)
_= plt.hist(harvester_grid['dist']) 
plt.show()

### Save the file

In [ ]:
harvester_grid.to_csv('planter-harvester-python.csv', index=False)